In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
216,AFG,Asia,Afghanistan,2020-08-13,37345.0,76.0,1354.0,10.0,959.327,1.952,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
374,ALB,Europe,Albania,2020-08-13,6817.0,141.0,208.0,3.0,2368.823,48.996,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
596,DZA,Africa,Algeria,2020-08-13,36699.0,495.0,1333.0,11.0,836.901,11.288,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-12,France,207794.0,0.0,30390.0,0.0,0.0,0.0
0,2020-08-12,China,89039.0,0.0,4700.0,0.0,0.0,0.0
0,2020-08-12,Italy,251591.0,0.0,35241.0,0.0,0.0,0.0
0,2020-08-12,Spain,333508.0,0.0,28643.0,0.0,0.0,0.0
0,2020-08-12,United States,5230318.0,0.0,165294.0,0.0,0.0,0.0
0,2020-08-12,World,20801680.0,0.0,749569.0,0.0,0.0,0.0
0,2020-08-12,United Kingdom,313775.0,0.0,46548.0,0.0,0.0,0.0
0,2020-08-12,Germany,219428.0,0.0,9217.0,0.0,0.0,0.0
0,2020-08-12,Iran,335061.0,0.0,19031.0,0.0,0.0,0.0
0,2020-08-12,Turkey,244239.0,0.0,5886.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-13,France,207794.0,206696.0,30390.0,30371.0,1098.0,19.0
0,2020-08-13,China,89039.0,89045.0,4700.0,4697.0,-6.0,3.0
0,2020-08-13,Italy,251591.0,251713.0,35241.0,35225.0,-122.0,16.0
0,2020-08-12,Spain,333508.0,329784.0,28643.0,28579.0,3724.0,64.0
0,2020-08-13,United States,5230318.0,5197148.0,165294.0,166027.0,33170.0,-733.0
0,2020-08-13,World,20801680.0,20614014.0,749569.0,749444.0,187666.0,125.0
0,2020-08-13,United Kingdom,313775.0,313798.0,46548.0,46706.0,-23.0,-158.0
0,2020-08-13,Germany,219428.0,219964.0,9217.0,9211.0,-536.0,6.0
0,2020-08-13,Iran,335061.0,333699.0,19031.0,18988.0,1362.0,43.0
0,2020-08-13,Turkey,244239.0,244392.0,5886.0,5891.0,-153.0,-5.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")